# Advanced Challenge: Randomized Benchmarking

In this challenge, you will implement randomised benchmarking, a protocol to determine the 
average gate fidelity of a quantum process.



In [1]:
from pyquil import Program
from pyquil.api import get_qc, WavefunctionSimulator, local_qvm
from pyquil.gates import *
from pyquil.noise import append_kraus_to_gate
import numpy as np
import os, inspect, sys
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, 'tests/')
sys.path.insert(0, 'auxiliary_unitaries/')

from auxiliary_functions.auxiliary_noise import define_noisy_cliffords, override_cliffords_in_circuit

%matplotlib inline

make_wf = WavefunctionSimulator()


In [2]:
qc_name = '2q-qvm'
with local_qvm():
    qc = get_qc(qc_name)

qubits = qc.qubits()

In [3]:
#Firstly define the pulse lengths:
    
# lengths = [5, 10, 20, 30, 50, 100] 
lengths = [2] 

## Randomized Benchmarking Protocol: Ideal Case

In [4]:
import random

def random_clifford(circuit, clifford_gates,random_seed):
    
    random.seed(random_seed)
    random_clifford_gate = random.choice(clifford_gates)

    circuit += random_clifford_gate(qubits[0])
    
    return circuit, random_clifford_gate


Next, define a function to apply a sequence of these random unitaries to the initial state

In [5]:
def random_clifford_sequence(circuit, random_seed, clifford_gates, length):
    
    random_seed = random_seed
    random_gates = []
    for length in range(length):
        circuit, random_clifford_gate = random_clifford(circuit, clifford_gates, random_seed*length)
        random_gates.append(random_clifford_gate)     # keep track of gate, run same sequence

    for gate in random_gates[::-1]: # Invert sequence and apply gates in reverse order (clifford are self-inverse)
        circuit += gate(qubits[0])
    return circuit


Now take that circuit of random cliffords, and measure the fidelity of the final state remaining in the initial state. 

The final state is denoted $|\psi\rangle$, and the initial state was the computational basis state, $|0\rangle$.

Therefore the fidelity is:

\begin{align}
F = |\langle 0 |\psi \rangle|^2 = \Pr(final state = |0\rangle)
\end{align}

So we can estimate this by measuring the qubit a number of times, and counting the number of times we get the '0' ($|0\rangle$) outcome.

# Task:
## Apply a measurement on the single qubit and compute the fidelity.

In [6]:
# def compute_fidelity(circuit, random_seed, clifford_gates, length, num_shots):
#     print('\n------------------------------------------\nRandom seed is', random_seed)
#     circuit = random_clifford_sequence(circuit, random_seed, clifford_gates, length)
#     print(circuit)
#     ro = circuit.declare('ro', 'BIT', 2)
#     circuit += MEASURE(qubits[0], ro[0])

#     circuit.wrap_in_numshots_loop(num_shots)
#     with local_qvm():
#         wf = make_wf.wavefunction(circuit)
#     print('Wavefunction is:', wf)
    
#     executable = qc.compile(circuit)
#     results = qc.run(executable)
#     print('Outcomes are',results)
#     prob_zero = list(results).count([0])/num_shots 

#     print(prob_zero)
    
#     return prob_zero


# # circuit = Program()
# # clifford_gates = [H, Z, X]

# # random_seed = 1
# # length = 4
# # num_shots = 1000

# # fidelity = compute_fidelity(circuit, random_seed, clifford_gates, length, num_shots)

# # print(fidelity)

In [7]:
# def compute_avg_fidelity(random_seeds, clifford_gates, lengths, num_shots, noise=False, p=0.1):
#     average_fidelities = np.zeros(len(lengths)) # Array of average fidelities for each sequence length
    
#     num_seq_repeats = len(random_seeds)
#     for length in range(len(lengths)):
#         fidelity = np.zeros(num_seq_repeats)
#         seq_len = lengths[length]
#         for repeat in range(num_seq_repeats):
#             circuit = Program()
#             if noise: # IGNORE THIS FOR NOW, we will add it later
#                 noisy_cliffords = define_noisy_cliffords(p)
#                 circuit = override_cliffords_in_circuit(circuit, noisy_cliffords)
#                 fidelity[repeat] = compute_fidelity(circuit, repeat, clifford_gates, seq_len, num_shots)
# #                 print(circuit)
# #                 print('This is happening')
#             else:
#                 fidelity[repeat] = compute_fidelity(circuit, repeat, clifford_gates, seq_len, num_shots)
#         average_fidelities[length] = (1/num_seq_repeats)*np.sum(fidelity)
    
#     return average_fidelities

# # sequence_length_repeats = 10 # Number of repeats of a particular sequence length

# # random_seeds = np.arange(0, 10, sequence_length_repeats)
# # num_shots = 10
# # clifford_gates = [Z, X]

# # average_fidelities = compute_avg_fidelity(random_seeds, clifford_gates, lengths, num_shots)
# # print('The sequence lengths are:', lengths)
# # print('\nThe average fidelities are:' , average_fidelities)

Next, we can plot the average fidelity as a function of the quantum circuit sequence lengths.
In the ideal case (that you have just done), clearly the average fidelity should be the same regardless of the sequence length.

When the inverse sequence of gates was applied, there was no noise, so effectively nothing happened to the $|0\rangle$ state; it returned to the $\0\rangle$ state **every** time, and the fidelity should always be $1$.

Increasing the length of the circuit does not affect this.

We can plot the average fidelities as a function of the sequence lengths (this is a trivial task in this case, but it will become relevant shortly.)

In [8]:
# plt.plot(lengths, average_fidelities)

Next, we must define 'noisy' gates to benchmark.

These will be noisy versions of the perfect clifford gates we used above.

For example, imagine our sequence of (one) random cliffords was $X$ applied the the input state:

\begin{align}
X|0\rangle
\end{align}

The corresponding density matrix would be (in the perfect case):

\begin{align}
\rho_{ideal} = X|0\rangle\langle 0|X
\end{align}

Now we can use the density matrix formalism to describe an ***error channel***. Firstly we can examine what is called a '*dephasing*' error channel, which means an erroneous phase error is applied to the state with some probability.

In other words, after we thought we had applied the $X$ gates to the state, because of the presence of this error channel, the *actual* state will be the correct state, with some probability, $p$, and some other state (with an error), with the remaining probability (1-p):

\begin{align}
\rho_{noisy} = \begin{cases}
\rho =  X|0\rangle\langle 0|X  &\text{ with some probability } p\\
Z\rho Z = ZX|0\rangle\langle 0|XZ &\text{ with probability } 1-p
\end{cases}
\end{align}

We can represent this by a *statistical mixture of these two possibilities*:
\begin{align}
\rho_{noisy} = \mathcal{E}_{dephase}(\rho_{ideal}) =  p\rho_{ideal}+ (1-p)Z\rho_{ideal} Z = pX|0\rangle\langle 0|X+ (1-p)ZX|0\rangle\langle 0|XZ 
\end{align}

where $\mathcal{E}_{dephase}(\rho_{ideal})$ is the dephasing error channel applied to the *ideal* state, $\rho_{ideal}$.

In the simplest example, we can assume this **same** error channel, $\mathcal{E}_{dephase}$  with the **same** error strength, $p$, is applied after ***every*** random gate. 

We can then use apply the same protocol as above with the noisy clifford gates (with this dephasing error model)
to get an idea of the average fidelity of the process.

*Hint: This should **not** be $1$ for all sequence lengths in this case, we should see the fidelity decay as longer sequences are applied $\rightarrow$ the effect of noise compounds after each gate.*

Of course, when using the simulator, we must **artificially** define our noisy gates. We will directly use the example [here](http://docs.rigetti.com/en/stable/noise.html) to create the dephasing channel.

This uses the idea of Kraus maps, which is a generalisation of the above, for our purposes, a kraus map is just defined by a matrix, for example $Z$ or the identity $I$ (which would be the ideal case).

The above equation can be written as:

\begin{align}
\rho_{noisy} = \mathcal{E}_{dephase}(\rho_{ideal}) =  \sqrt{p}\mathbb{I}\rho_{ideal} \sqrt{p}\mathbb{I}+ \sqrt(1-p)Z\rho_{ideal} \sqrt(1-p)Z 
\end{align}


In [9]:
# # Define function to return the operators sqrt(p)I, sqrt(1-p)Z
# def dephasing_kraus_map(p):
#     """
#     Generate the Kraus operators corresponding to a dephasing channel.

#     :params float p: The one-step dephasing probability.
#     :return: A list [k1, k2] of the Kraus operators that parametrize the map.
#     :rtype: list
#     """
#     return [np.sqrt(1-p)*np.eye(2), np.sqrt(p)*np.diag([1, -1])]

Now we must apply this channel to the target gate, which must be done for each of the Clifford gates.
The example for the X gate is below:

*Note: This is only done for this example, you will not use the program 'test_circuit' afterwards.*

In [10]:
# p = .1
# corrupted_X = append_kraus_to_gate(dephasing_kraus_map(p), np.array([[0, 1], [1, 0]]))

# test_circuit = Program()
# noisy_x =  test_circuit.define_noisy_gate("X", [0], corrupted_X)

# test_circuit += X(0)
# # noisy_x =  test_circuit.define_noisy_gate("X", [0], corrupted_X)
# print(test_circuit)
# from auxiliary_functions.auxiliary_noise import define_noisy_cliffords, override_cliffords_in_circuit


We can load the sequence of predefined noisy clifford gates from an auxiliary function:

In [11]:
from pyquil import Program
from pyquil.api import get_qc, WavefunctionSimulator, local_qvm
from pyquil.gates import *
from pyquil.noise import append_kraus_to_gate
import numpy as np

make_wf = WavefunctionSimulator()
qc_name = '2q-qvm'
with local_qvm():
    qc = get_qc(qc_name)

qubits = qc.qubits()

def dephasing_kraus_map(p):

    return [np.sqrt(1-p)*np.eye(2), np.sqrt(p)*np.array([[0, 1], [1, 0]])]


p = 1
prog = Program()

corrupted_H = append_kraus_to_gate(dephasing_kraus_map(p), 1/np.sqrt(2)*np.array([[1, 1], [1, -1]]))

prog.define_noisy_gate("H", [0], corrupted_H)

prog += H(0)
prog += H(0)

num_shots = 5

ro = prog.declare('ro', 'BIT', 1)
prog += MEASURE(qubits[0], ro[0])
prog.wrap_in_numshots_loop(num_shots)
print(prog)

executable = qc.compile(prog)
results = qc.run(executable)

with local_qvm():
    wf = make_wf.wavefunction(prog)
    
print('Wavefunction is:', wf)
print('Outcomes are',results)


PRAGMA ADD-KRAUS H 0 "(0.0 0.0 0.0 0.0)"
PRAGMA ADD-KRAUS H 0 "(0.7071067811865475 -0.7071067811865475 0.7071067811865475 0.7071067811865475)"
H 0
H 0
DECLARE ro BIT[1]
MEASURE 0 ro[0]

Wavefunction is: (1+0j)|1>
Outcomes are [[0]
 [0]
 [0]
 [0]
 [0]]


In [12]:

# sequence_length_repeats = 2 # Number of repeats of a particular sequence length

# random_seeds = np.arange(0, 2, sequence_length_repeats)
# num_shots = 2
# clifford_gates = [Z, H]

# average_fidelities = compute_avg_fidelity(random_seeds, clifford_gates, lengths, num_shots, noise=True, p=0.5)
# print('The sequence lengths are:', lengths)
# print('\nThe average fidelities are:' , average_fidelities)